In [1]:
import json
import os 
import re 
import time

import cv2
from   matplotlib import pyplot as plt
import numpy as np
from   PIL import Image, ImageDraw, ImageFont
import tensorflow as tf
import tensorflow_datasets as tfds

In [7]:
IMG_DIR = '../../../img'
DATA = f'{IMG_DIR}/bounding_box'

In [8]:
def draw_bounding_box(
        img, xmin, ymin, xmax, ymax, color=(255, 0, 0), thickness=5):
    '''
    Adds a bounding box to an image.
    
    Args:
      image: a PIL.Image object.
      xmin, ymin, xmax, ymax: limits of bounding box.
      color: color to draw bounding box.
      thickness: line thickness.
    '''
    img_w, img_h = img.shape
    cv2.rectangle(img, 
                  (int(xmin), int(ymin)), 
                  (int(xmax), int(ymax)), 
                  color,
                  thickness)

In [9]:
def draw_bounding_boxes(img, boxes, color=[], thickness=5):
    '''
    Draws bounding boxes on image.
    
    Args:
      image: a PIL.Image object.
      boxes: a 2 dimensional numpy array of [N, 4]: 
        (xmin, ymin, xmax, ymax).
        The coordinates are in normalized format between [0, 1].
      color: color to draw bounding box
      thickness: line thickness
                           
    Raises:
      ValueError: if boxes is not a [N, 4] array
    '''
    boxes_shape = boxes.shape
    if not boxes_shape:
        return
    if len(boxes_shape) != 2 or boxes_shape[1] != 4:
        raise ValueError('boxes must be of size [N, 4]')
    for i in range(boxes_shpae[0]):
        draw_bounding_box(img, 
                          boxes[i, 0], 
                          boxes[i, 1],
                          boxes[i, 2],
                          boxes[i, 3],
                          color[i],
                          thickness)

In [11]:
def draw_bounding_boxes_on_img_array(img, boxes, color=[], thickness=5):
    '''
    Draws bounding boxes on image (numpy array).

    Args:
      image: a numpy array object.
      boxes: a 2 dimensional numpy array of [N, 4]: 
        (xmin, ymin, xmax, ymax).
        The coordinates are in normalized format between [0, 1].
      color: color to draw bounding box
      thickness: line thickness
    
    Raises:
      ValueError: if boxes is not a [N, 4] array
    '''
    draw_bounding_boxes(img, boxes, color, thickness)
    return img